# Run original and READII negative control CTs through FMCIB

This notebook is set up to use outputs from the `run_readii_prep.ipynb` notebook. 

CT images cropped around a Gross Tumour Volume (GTV), resized, and saved as nifti files are fed into an instance of FMCIB for inference and all features are saved.

## Set up pixi environment kernel

1. Run the following commands in the terminal:

    ```bash
    $ pixi install

    $ pixi shell -e fmcib

    $ ipython kernel install --user --name=fmcib --display-name=fmcib
    ```

2. In the `Select Kernel` menu at the top right of the notebook, select `Jupyter Kernel` as the source. 

3. Refresh the options and one called `fmcib` should appear. Select this option.

## Imports

In [2]:
import yaml
from pathlib import Path

import sys; sys.path.append("code")
from infer import runFMCIBInfer

## Initialize dataset name and negative control settings

In [3]:
config = yaml.load(open("config/RADCURE.yaml", "r"), Loader=yaml.FullLoader)

DATASET_NAME = config["dataset_name"]

NEG_CONTROL_REGIONS = config["negative_control_regions"]
NEG_CONTROL_TYPES = config["negative_control_types"]

## Set up data directories

In [5]:
# Path to dir containing fmcib input csvs
fmcib_csv_dir = Path("procdata", DATASET_NAME, "images/fmcib_input")

## Run each image type and crop method through FMCIB and save out the resulting features

This will run on all crop methods present the `procdata/images/fmcib/` directory.

In [ ]:
for crop_type_dir_path in fmcib_csv_dir.glob("*/"):
    crop_type = crop_type_dir_path.name
    # Set up results directory
    features_dir = Path("results", DATASET_NAME, "fmcib_features", crop_type)
    Path.mkdir(features_dir, parents=True, exist_ok=True)

    for fmcib_csv_path in crop_type_dir_path.glob("*.csv"):
        feature_file_save_path = Path(features_dir, f"fmcib_features_{fmcib_csv_path.name.removeprefix('fmcib_input_')}")
        
        # Only run inference if the output does not already exist
        if not feature_file_save_path.exists():
            runFMCIBInfer(feature_file_save_path=str(feature_file_save_path), 
                        csv_path=str(fmcib_csv_path), 
                        weights_path="model_weights.torch",
                        precropped=True)